<a href="https://colab.research.google.com/github/Noam-Coh3n/ModCrypto/blob/main/Security_Proof_of_Pseudo_OTP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Modern Cryptography 2022/23, University of Amsterdam. Teacher: Christian Schaffner* 
# The Security of Pseudo-OTP

## Preliminaries

Install the [Python bitstring module](https://pythonhosted.org/bitstring/contents.html).

In [29]:
!pip install bitstring

from bitstring import Bits, BitArray
from tqdm import tqdm
from typing import Callable, Tuple
import secrets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [30]:
# for n=10 a unary encoding of 1^n is 
Bits(int=-1, length=10)

Bits('0b1111111111')

## A particular (bad) PRG
This PRG is bad, as it simply outputs the seed twice, one can easily distinguish this output from fully random strings.

In [31]:
def G(s: Bits) -> Bits:
    """
    the pseudo-random generator G(s) = s || s
    :param s: seed s
    :return: the concatenation of s with itself
    """
    return s + s

## Pseudo One-Time Pad
The Pseudo One-Time Pad uses the (bad) PRG above to encrypt (and decrypt) messages (or ciphertexts) by XORing the output of the PRG with the message (or ciphertext).

In [62]:
class PseudoOTP(object):
    """ Pseudo OTP """
    @staticmethod
    def Gen(unary_n: Bits) -> Bits:
        """
        takes as input 1^n (the security parameter written in unary) and outputs a key k
        :param unary_n: an (arbitrary) input of length n
        :return: a random key of length at least n
        """
        n = len(unary_n)
        key = Bits(uint=secrets.randbits(n), length=n)
        return key

    @staticmethod
    def Enc(k: Bits, m: Bits) -> Bits:
        """
        encrypts message m with key k
        :param k: a key k
        :param m: the message to be encrypted
        :return: the ciphertext c
        """
        assert len(k) * 2 == len(m), 'the length of message has to be twice the length of the key'
        return G(k) ^ m

    @staticmethod
    def Dec(k: Bits, c: Bits) -> Bits:
        """
        decrypts ciphertext c with key k
        :param k: a key k
        :param m: the ciphertext to be decrypted
        :return: the message m
        """
        assert len(k) * 2 == len(c), 'the length of the ciphertext has to be twice the length of the key'
        return G(k) ^ c

## Generic attacker playing in the $\mathsf{PrivK}^{\mathsf{eav}}_{\mathcal{A}, \Pi}$ game


In [63]:
class Adversary(object):
    """ adversary playing in PrivK^eav game """
    @staticmethod
    def challenge_plaintexts(unary_n: Bits) -> Tuple[Bits, Bits]:
        """
        The first part of the adversary receives the security parameter n and returns two challenge plaintexts
        of the same length
        :param unary_n: unary encoding of security parameter
        :return: (m0, m1) two challenge plaintexts of the same length
        """
        n = len(unary_n)  # security parameter
        allzero = Bits(uint=0, length=2*n)
        return allzero, allzero

    @staticmethod
    def guess_bit(c: Bits) -> int:
        """
        The second part of the adversary receives the challenge ciphertext from the challenger and has to guess which
        of its two challenge plaintexts was encrypted
        :param c:
        :return: a guess of which of the two plaintexts were encrypted to c
        """
        n = int(len(c) / 2)
        return 0

## The $\mathsf{PrivK}^\mathsf{eav}$ game

In [67]:
def PrivK(Pi: PseudoOTP, Adv: Adversary, n: int) -> bool:
    """
    Private-key security game played between a challenger and adversary
    1. Adv is given the security parameter and comes up with two challenge plaintexts m_0 and m_1
    2. Challenger generates a new secret key and picks a random bit b
    3. Chall encrypts message m_b into challenge ciphertext c
    4. Based on c, Adv has to guess the bit b in order to win the game

    :param Pi: private-key encryption scheme
    :param Adv: adversary
    :param n: security parameter
    :return: a Boolean value whether the Adversary has won the game or not
    """

    # 1. Adv is given the security parameter and comes up with two challenge plaintexts m_0 and m_1
    m_0, m_1 = Adv.challenge_plaintexts(Bits(int=-1, length=n))

    # 2. Challenger generates a new secret key and picks a random bit b
    key = Pi.Gen(Bits(int=-1, length=n))
    b = secrets.randbelow(2)

    # 3. Challenger encrypts message m_b into challenge ciphertext c
    if b == 0:
        c = Pi.Enc(key, m_0)
    elif b == 1:
        c = Pi.Enc(key, m_1)

    # 4. Based on c, Adv has to guess the bit b in order to win the game
    b_guess = Adv.guess_bit(c)

    return b == b_guess


def test_adversary(Pi: PseudoOTP, Adv: Adversary, n: int, nr_runs: int) -> int:
    """
    tests how well Adv does when playing in the PrivK security game by executing the game nr_runs times
    and taking statistics
    :param Pi: private-key encryption scheme
    :param A: adversary
    :param n: security parameter
    :param nr_runs: number of runs of the PrivK_{Pi, Adv}(1^n) game
    :return: the number of wins
    """
    wins = 0
    for i in tqdm(range(nr_runs)):
        if PrivK(Pi, Adv, n):
            wins += 1

    print("\nout of {} runs, the adversary has won {}".format(nr_runs, wins))
    return wins

# Build a succesful adversary!
**adjust the adversary below so that the following assertion holds, and explain how it works!**

In [69]:
class OTP_Adversary(Adversary):
    """ adversary playing in PrivK^eav game against Pseudo OTP """
    @staticmethod
    def challenge_plaintexts(unary_n: Bits) -> Tuple[Bits, Bits]:
        """
        The first part of the adversary receives the security parameter n and returns two challenge plaintexts
        of the same length
        :param unary_n: unary encoding of security parameter
        :return: (m0, m1) two challenge plaintexts of the same length
        """
        n = len(unary_n)  # security parameter
        allzero = Bits(uint=0, length=2*n)
        other = Bits(uint=0, length=2*n-1) + Bits(uint=1, length=1)
        return allzero, other

    @staticmethod
    def guess_bit(c: Bits) -> int:
        """
        The second part of the adversary receives the challenge ciphertext from the challenger and has to guess which
        of its two challenge plaintexts was encrypted
        :param c:
        :return: a guess of which of the two plaintexts were encrypted to c
        """
        n = int(len(c) / 2)
        return c[:n] != c[n:]

assert test_adversary(PseudoOTP, OTP_Adversary, 5, 1000) > 700, 'build a better adversary!'


100%|██████████| 1000/1000 [00:00<00:00, 7924.22it/s]


out of 1000 runs, the adversary has won 1000


## Security Reduction

The security proof of Theorem 3.18 in [KL] (also described in [this video](https://canvas.uva.nl/courses/32076/files/7071821)) gives an explicit way how to turn a successful PrivK^eav attacker into a successful distinguisher of the PRG G. Let's see it in action!

In [70]:
def DistFromAdv(Adv: Adversary, w: Bits) -> bool:
    '''
    Given a successful attacker of the PseudoOTP scheme, we define a successful distinguisher of the PRG. See proof of
    Theorem 3.18 in [KL] and video: https://canvas.uva.nl/courses/32076/files/7071821
    :param Adv: a PrivK attacker on the PseudoOTP scheme with PRG G
    :param w: input to the PRG
    :param n: security parameter
    :return: the distinguisher's output bit
    '''
    # we play the PrivK game towards the adversary

    n = int(len(w)/2)  # security parameter, length of plaintexts

    # 1. Adv is given the security parameter and comes up with two challenge plaintexts m_0 and m_1
    m_0, m_1 = Adv.challenge_plaintexts(Bits(int=-1, length=n))

    # 2. Challenger generates a new secret key and picks a random bit b
    b = secrets.randbelow(2)

    # 3. Instead of encrypting, we pad the message m_b with w
    if b == 0:
        c = m_0 ^ w
    elif b == 1:
        c = m_1 ^ w

    # 4. Based on c, Adv has to guess the bit b in order to win the game
    b_guess = Adv.guess_bit(c)

    return b == b_guess

## Quality check of a PRG distinguisher

In [71]:
def compute_probability_difference(G: Callable[[Bits], Bits], D: Callable[[Bits], bool], n: int) -> float:
    """
    Def 3.14 [KL]: A PRG G is secure if no PPT distinguisher D can distinguish an output from G (with random seed) from
    a fully uniform output. Formally, we should have that
    | Pr_{s <- {0,1}^n} [ D( G(s) ) = 1 ] - Pr_{w <- {0,1}^G.l_out(n)} [ D(w) = 1 ] | < negl(n)

    For small parameters of n, we can brute-force compute these probabilities and output the absolute difference
    :param G: PRG candidate function
    :param D: PRG distinguisher
    :param n: security parameter
    :return: the absolute difference in probability
    """

    print("Computing ")

    l_out = len(G(Bits(uint=0, length=n))) # feed 0^n to G and measure length of the output
    assert l_out > n, 'PRG has to be length-expanding'
    assert l_out <= 20, 'for output lengths l_out larger than 20, the brute-force computation of the probabilities will take too long'

    # compute Pr_{s <- {0,1}^n} [ D( G(s) ) = 1 ]
    counter = 0
    for s in tqdm(range(0, 2**n)):
        w = G(Bits(uint=s, length=n))
        if D(w):
            counter += 1
    pr_g = counter / (2 ** n)

    # compute Pr_{w <- {0,1}^G.l_out(n)} [ D(w) = 1 ]
    counter = 0
    for w in tqdm(range(0, 2**l_out)):
        r = Bits(uint=w, length=l_out)
        if D(r):
            counter += 1
    pr_w = counter / (2 ** l_out)

    # output difference
    print("\n\n n: {}, l_out: {}".format(n, l_out))
    print('Pr_{{s <- {{0,1}}^{}}} [ D( G(s) ) = 1 ] is {}'.format(n, pr_g))
    print("Pr_{{w <- {{0,1}}^{}}} [ D(w) = 1 ] is {}".format(l_out, pr_w))

    print('absolute difference: {}'.format(abs(pr_g - pr_w)))
    return abs(pr_g - pr_w)

Observe that a succesful PrivK^eav attacker also gives a successful PRG distinguisher

In [72]:
n = 6
assert compute_probability_difference(G, lambda w: DistFromAdv(OTP_Adversary, w), n) > 0.3

Computing 


100%|██████████| 4096/4096 [00:00<00:00, 10310.72it/s]



 n: 6, l_out: 12
Pr_{s <- {0,1}^6} [ D( G(s) ) = 1 ] is 1.0
Pr_{w <- {0,1}^12} [ D(w) = 1 ] is 0.501953125
absolute difference: 0.498046875


while the generic PrivK^eav adversary (that always outputs 0) does not yield a good PRG distinguisher

In [73]:
n = 6
compute_probability_difference(G, lambda w: DistFromAdv(Adversary, w), n) > 0.3

Computing 


100%|██████████| 4096/4096 [00:00<00:00, 10677.55it/s]



 n: 6, l_out: 12
Pr_{s <- {0,1}^6} [ D( G(s) ) = 1 ] is 0.546875
Pr_{w <- {0,1}^12} [ D(w) = 1 ] is 0.49853515625
absolute difference: 0.04833984375


False